# Final project: Dota winner prediction
# by Anastasia Shukhova 


In [1]:
import pandas
import numpy as np
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn import grid_search
import time
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import collections
import warnings

Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [65]:
features = pandas.read_csv('./features.csv', index_col='match_id',  dtype = 'float64')

features = features.drop(['duration','radiant_win', 'tower_status_radiant',
                          'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1)

features_test = pandas.read_csv('./features_test.csv', index_col = 'match_id',  dtype = 'float64')


In [66]:
features_test.columns.values

array(['start_time', 'lobby_type', 'r1_hero', 'r1_level', 'r1_xp',
       'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths', 'r1_items', 'r2_hero',
       'r2_level', 'r2_xp', 'r2_gold', 'r2_lh', 'r2_kills', 'r2_deaths',
       'r2_items', 'r3_hero', 'r3_level', 'r3_xp', 'r3_gold', 'r3_lh',
       'r3_kills', 'r3_deaths', 'r3_items', 'r4_hero', 'r4_level', 'r4_xp',
       'r4_gold', 'r4_lh', 'r4_kills', 'r4_deaths', 'r4_items', 'r5_hero',
       'r5_level', 'r5_xp', 'r5_gold', 'r5_lh', 'r5_kills', 'r5_deaths',
       'r5_items', 'd1_hero', 'd1_level', 'd1_xp', 'd1_gold', 'd1_lh',
       'd1_kills', 'd1_deaths', 'd1_items', 'd2_hero', 'd2_level', 'd2_xp',
       'd2_gold', 'd2_lh', 'd2_kills', 'd2_deaths', 'd2_items', 'd3_hero',
       'd3_level', 'd3_xp', 'd3_gold', 'd3_lh', 'd3_kills', 'd3_deaths',
       'd3_items', 'd4_hero', 'd4_level', 'd4_xp', 'd4_gold', 'd4_lh',
       'd4_kills', 'd4_deaths', 'd4_items', 'd5_hero', 'd5_level', 'd5_xp',
       'd5_gold', 'd5_lh', 'd5_kills', 'd5_death

Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [67]:
print features.count()[features.count() < 97230]
# first_blood_time/team: могло так получиться, что в течение первых пяти минут никто никого убить не успел 
# radiant_bottle_time: в течение первых пяти минут кто-то мог не купить bottle 

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64


Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой

In [68]:
features_logit = features.fillna(value = 0)
features_tree = features.fillna(value = 99999)

Какой столбец содержит целевую переменную? Запишите его название.

In [71]:
target = pandas.read_csv('./features.csv', index_col='match_id', dtype = 'float64')['radiant_win']
# 'radiant_win'
print target.size


97230


Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [72]:
start_time = datetime.datetime.now()

cv = KFold(target.size, n_folds=5, shuffle=True, random_state=241)
grid = {'n_estimators': np.arange(10, 100, 10)}
gradboost = GradientBoostingClassifier(random_state = 241)
gs = grid_search.GridSearchCV(gradboost, grid, scoring = 'roc_auc', cv = cv, n_jobs = -1)
gs.fit(X = features_tree, y = target)
for score in gs.grid_scores_:
    print score.mean_validation_score, score.parameters

print 'Time elapsed:', datetime.datetime.now() - start_time

0.666472909281 {'n_estimators': 10}
0.68270790674 {'n_estimators': 20}
0.689413581011 {'n_estimators': 30}
0.694317561957 {'n_estimators': 40}
0.697282662316 {'n_estimators': 50}
0.700145548868 {'n_estimators': 60}
0.702162727796 {'n_estimators': 70}
0.704104006541 {'n_estimators': 80}
0.705542216261 {'n_estimators': 90}
Time elapsed: 0:17:50.648734


# Для отчета Boosting Classifier 


Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?


In [73]:
print features.count()[features.count() < 97230]
# first_blood_time/team: могло так получиться, что в течение первых пяти минут никто никого убить не успел 
# radiant_bottle_time: в течение первых пяти минут кто-то мог не купить bottle 

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64


`first_blood_time/team`: могло так получиться, что в течение первых пяти минут никто никого убить не успел 
`radiant_bottle_time`: в течение первых пяти минут кто-то мог не купить `bottle`

Как называется столбец, содержащий целевую переменную?


`radiant_win`

Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями?  Какое качество при этом получилось?


In [74]:
start_time = datetime.datetime.now()

cv = KFold(target.size, n_folds = 5, shuffle = True, random_state = 241)
clf = GradientBoostingClassifier(n_estimators = 30)
clf.fit(X = features_tree, y = target)
scores = cross_val_score(clf, X = features_tree, y = target, cv = cv, scoring = 'roc_auc', n_jobs = -1)
print np.mean(scores)
print 'Time elapsed:', datetime.datetime.now() - start_time
#0:02:02.156063
# 0:01:41.173742 with paralell

0.689413581011
Time elapsed: 0:01:48.204344


Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? 


#### Нет, пожалуй, в увеличении числа деревьев смысла нет. После 30 деревьев качество модели увеличивается, но совсем медленно, в то время как время вычислений значительно увеличивается 

Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

#### распаралеллить 
#### ограничить глубину деревьев 
#### уменьшить число наблюдений, случайным образом удалив часть наблюдений

# Ridge 


Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [75]:
features_logit_norm = preprocessing.StandardScaler().fit_transform(features_logit)
features_logit_norm = pandas.DataFrame(features_logit_norm)
features_logit_norm.columns = features_logit.columns

In [76]:
warnings.filterwarnings('ignore')

start_time = datetime.datetime.now()

cv = KFold(target.size, n_folds=5, shuffle=True, random_state=241)
grid = {'C': np.power(10.0, np.arange(-5, 6))}
logit = LogisticRegression(random_state = 241)
gs = grid_search.GridSearchCV(logit, grid, scoring = 'roc_auc', cv = cv, n_jobs = -1)
gs.fit(X = features_logit_norm, y = target)
for score in gs.grid_scores_:
    print score.mean_validation_score, score.parameters

print 'Time elapsed:', datetime.datetime.now() - start_time

0.695120378788 {'C': 1.0000000000000001e-05}
0.711250115451 {'C': 0.0001}
0.716180247426 {'C': 0.001}
0.716341465365 {'C': 0.01}
0.716310086833 {'C': 0.10000000000000001}
0.716306583646 {'C': 1.0}
0.716306337841 {'C': 10.0}
0.716306265779 {'C': 100.0}
0.716306261535 {'C': 1000.0}
0.716306256242 {'C': 10000.0}
0.716306256242 {'C': 100000.0}
Time elapsed: 0:01:31.408049


Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [77]:
features_logit_norm = pandas.DataFrame(features_logit_norm)
features_logit_norm.columns = features_logit.columns
features_nocat = features_logit_norm.drop(['lobby_type','r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                          'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)

start_time = datetime.datetime.now()

cv = KFold(target.size, n_folds=5, shuffle=True, random_state=241)
grid = {'C': np.power(10.0, np.arange(-5, 6))}
logit = LogisticRegression(random_state = 241)
gs = grid_search.GridSearchCV(logit, grid, scoring = 'roc_auc', cv = cv, n_jobs = -1)
gs.fit(X = features_nocat, y = target)
for score in gs.grid_scores_:
    print score.mean_validation_score, score.parameters

print 'Time elapsed:', datetime.datetime.now() - start_time

0.695056932989 {'C': 1.0000000000000001e-05}
0.711248389557 {'C': 0.0001}
0.716235591021 {'C': 0.001}
0.716400950653 {'C': 0.01}
0.716373784472 {'C': 0.10000000000000001}
0.716370753719 {'C': 1.0}
0.716370478246 {'C': 10.0}
0.71637049733 {'C': 100.0}
0.716370531224 {'C': 1000.0}
0.716370531226 {'C': 10000.0}
0.716370531226 {'C': 100000.0}
Time elapsed: 0:01:12.441284


На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [78]:
unique_heros = np.unique(features[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero','r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']])
print len(unique_heros)
print unique_heros

columns = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero','r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for column in columns:
    features[column] = features[column].replace([112, 109, 110], [107, 108, 24])

108
[   1.    2.    3.    4.    5.    6.    7.    8.    9.   10.   11.   12.
   13.   14.   15.   16.   17.   18.   19.   20.   21.   22.   23.   25.
   26.   27.   28.   29.   30.   31.   32.   33.   34.   35.   36.   37.
   38.   39.   40.   41.   42.   43.   44.   45.   46.   47.   48.   49.
   50.   51.   52.   53.   54.   55.   56.   57.   58.   59.   60.   61.
   62.   63.   64.   65.   66.   67.   68.   69.   70.   71.   72.   73.
   74.   75.   76.   77.   78.   79.   80.   81.   82.   83.   84.   85.
   86.   87.   88.   89.   90.   91.   92.   93.   94.   95.   96.   97.
   98.   99.  100.  101.  102.  103.  104.  105.  106.  109.  110.  112.]


Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [79]:
warnings.filterwarnings('ignore')

columns = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero','r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for column in columns:
    features[column] = features[column].replace([112, 109, 110], [107, 108, 24])
# N — количество различных героев в выборке
X_pick = np.zeros((features.shape[0], len(unique_heros)))
print X_pick.shape
for i, match_id in enumerate(features.index):
    #print 'new', i, match_id
    for p in xrange(5):
        #print features.ix[match_id, 'r%d_hero' % (p+1)]
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1


(97230, 108)


In [80]:
features_logit_norm = pandas.DataFrame(features_logit_norm)
features_logit_norm.columns = features_logit.columns
features_nocat = features_logit_norm.drop(['lobby_type','r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                          'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
new_features_logit = pandas.concat([features_nocat, pandas.DataFrame(X_pick)], axis = 1)


 Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [81]:
start_time = datetime.datetime.now()

cv = KFold(target.size, n_folds=5, shuffle=True, random_state=241)
grid = {'C': np.power(10.0, np.arange(-5, 6))}
logit = LogisticRegression(random_state = 241)
gs = grid_search.GridSearchCV(logit, grid, scoring = 'roc_auc', cv = cv, n_jobs = -1)
gs.fit(X = new_features_logit, y = target)
for score in gs.grid_scores_:
    print score.mean_validation_score, score.parameters

print 'Time elapsed:', datetime.datetime.now() - start_time

0.699171322745 {'C': 1.0000000000000001e-05}
0.725022147156 {'C': 0.0001}
0.746296235489 {'C': 0.001}
0.751735957912 {'C': 0.01}
0.75193748445 {'C': 0.10000000000000001}
0.751919575548 {'C': 1.0}
0.751917162612 {'C': 10.0}
0.751917210381 {'C': 100.0}
0.751916957161 {'C': 1000.0}
0.751917148904 {'C': 10000.0}
0.751916994262 {'C': 100000.0}
Time elapsed: 0:03:05.720098


Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [82]:
features_test = features_test.fillna(value = 0)
features_test_norm = preprocessing.StandardScaler().fit_transform(features_test)
features_test_norm = pandas.DataFrame(features_test_norm)
features_test_norm.columns = features_test.columns

features_nocat_test = features_test_norm.drop(['lobby_type','r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                          'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)

columns = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero','r5_hero','d1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for column in columns:
    features_test[column] = features_test[column].replace([112, 109, 110], [107, 108, 24])



# N — количество различных героев в выборке
X_pick = np.zeros((features_test.shape[0], len(unique_heros)))

for i, match_id in enumerate(features_test.index):
    #print 'new', i, match_id
    for p in xrange(5):
        #print features.ix[match_id, 'r%d_hero' % (p+1)]
        X_pick[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
new_features_test = pandas.concat([features_nocat_test, pandas.DataFrame(X_pick)], axis = 1)

In [83]:
logit_fin = LogisticRegression(C = 0.1, random_state = 241)
logit_fin.fit(new_features_logit, target)

probabilities = logit_fin.predict_proba(new_features_test)


In [84]:
print np.max(probabilities)
print np.min(probabilities)
print np.isnan(probabilities).any()


0.996459238405
0.003540761595
False
